# Tensorflow 2 implementation of open-neural-apc

Copyright (c) 2020, Nico Jahn <br>
All rights reserved.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q

# needed for the video output/investigation (not needed for training/testing)
#apt-get -qq update && apt-get -qq install -y ffmpeg

In [ ]:
from utils import loadConfig, prepare_env
prepare_env()
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = loadConfig(verbose=1)
# Since I saved the data as "uint8" and the sensor is usually placed at a height of 2 meters
#    the resolution should be just below 1cm
#    but since the noise frames at the end of most sequences produce larger values, this might not be the case
# The "accuracy error niveau" is the absolutely permissible difference so that a prediction 
#    on the label is still counted as correct (remember: it's a regression task)
# The "frame_stride" parameter indicates which frames are used from the original sequence
# The original sequences are at about 40 FPS. The model is trained with just 10 FPS
# The "safe_steps" parameter is used to safe the model every "safe steps" epochs

In [ ]:
# parsing command line arguments and overwriting config if those are supplied
from argument_parser import parse_arguments, overwrite_config
parsed_arguments, _ = parse_arguments()
overwrite_config(parsed_arguments, data_parameter, model_parameter, training_parameter)

In [ ]:
''' Loading the training data '''
from data_loader import DataLoader
training_data = DataLoader(training_parameter, data_parameter['data'], "training")

if len(training_data)>0:
    ''' Creating a model and save the config'''
    from napc import NeuralAPC
    # create model from config
    napc = NeuralAPC(model_parameter, training_parameter)
    napc.compile()
    napc.save()

    # writing config file into model folder
    import json
    new_config = {"data_parameter": data_parameter, "model_parameter": model_parameter, "training_parameter": training_parameter}
    config_path = napc.model_path + "config.json"
    with open(config_path, 'w+') as config:
        config.write(json.dumps(new_config, sort_keys=True, indent=2))
    print(f'Model folder created and config saved: {config_path}')

    '''Data generator initialization'''
    from data_generator import DataGenerator
    # this is class which preprocesses the training data every epoch
    # it creates the necessary labels/bounds and augments the data
    training_generator = DataGenerator(training_data, training_parameter)

    ''' Training procedure'''
    training_generator.on_epoch_end()
    napc.model.fit(training_generator, epochs=training_parameter['epochs'],
                   initial_epoch=napc.epoch, max_queue_size=training_generator.num_batches,
                   workers=4, use_multiprocessing=False, callbacks=napc.callbacks)

In [ ]:
''' For all of you which don't have the training data and just want to execute the notebooks '''
if len(training_data)==0:
    data_parameter, model_parameter, training_parameter = loadConfig('models/config.json',
                                                                     verbose=False)
    from napc import NeuralAPC
    napc = NeuralAPC(model_parameter, training_parameter, restored=True)
    # Loading the included model (it has no subdirectory)
    napc.loadModel(10000,'models/')
    # The model_path of the model is not 'models/', but the previously created subdirectory
    # You could now train it further/save it/ etc.

In [ ]:
''' Produce videos on all validation sequences or just validate the model '''
# de-/activate video creation
create_vids = False

# copy dict from training and modify the concatenation
validation_parameter = training_parameter.copy()
validation_parameter["concatenation_length"] = 1

# read the validation data
from data_loader import DataLoader
validation_data = DataLoader(validation_parameter, data_parameter['data'], "validation")

# process them (i need the bounds in y for the accuracy and the videos)
from data_generator import DataGenerator
validation_generator = DataGenerator(validation_data, validation_parameter)

# With training=False, no augmentation is applied to the input data
# Therefore, results are the closest possible real-world setting
validation_generator.on_epoch_end(training=False)

import numpy as np
from video_generator import createVideo
from tqdm.notebook import trange, tqdm

accuracy = []
for batch_idx in trange(validation_generator.num_batches, desc='Batches done', unit='batches'):
    # get batch, predict and calculate accuracy
    x,y = validation_generator[batch_idx]
    predictions = napc.model.predict_on_batch(x)
    accuracy += [napc.accuracy(y, predictions)]
    
    # creates my videos
    if create_vids:
        # has to create the videos for every element
        for sample_idx, prediction in enumerate(tqdm(predictions, desc='Videos created', leave=False, unit='videos')):
            output_dimensions = model_parameter['output_dimensions']
            
            # mask/remove the padding if batched
            binary_mask = (np.minimum(0, y[sample_idx,:,2*output_dimensions])+1).astype(bool)
            input_sequence = tf.boolean_mask(x[sample_idx], binary_mask, axis=0)
            pred = K.eval(tf.boolean_mask(prediction, binary_mask, axis=0))
            lower_bound = tf.boolean_mask(y[sample_idx,:,0:output_dimensions],
                                          binary_mask, axis=0)
            upper_bound = tf.boolean_mask(y[sample_idx,:,output_dimensions:2*output_dimensions],
                                          binary_mask, axis=0)

            # if creating the video takes to long, you can adjust the default dpi=300 parameter
            # napc.epoch and sample_idx are just used for the video name
            # the other arguments are actually plotted in the video
            video_index = validation_parameter["batch_size"]*batch_idx+sample_idx
            createVideo(napc.epoch, video_index, input_sequence, pred, lower_bound, upper_bound,
                        model_parameter['input_dimensions'], class_names=data_parameter["class_names"])

In [ ]:
# Since I'm not allowed to upload more sequences and I don't want to publish a perfect model
# (therefore, I haven't tested this one) the accuracy is just an approximation of the true capabilities
# The 'validation' data in this case is in fact a test set (last epoch was chosen without selection)
# In practice someone would use k-fold-CV and would reason about the average performance
import numpy as np
# So let's have a look how well the model does...
print(f'Accuracy: {100*np.mean(accuracy)} %')